In [ ]:
import psycopg2
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
class trip:
    def __init__(self, init_pos, end_pos, init_time, end_time):
        self.taxi_init_pos = init_pos
        self.taxi_end_pos = end_pos
        self.taxi_init_time = init_time
        self.taxi_end_time = end_time

    def bus_init(self, init_pos, init_stop, init_time, bus_id, trip_id):

        self.bus_init_pos = init_pos
        self.bus_init_stop = init_stop
        self.bus_init_time = init_time

        self.bus_id = bus_id
        self.trip_id = trip_id

    def bud_end(self, end_pos, end_stop, end_time):
        self.bus_end_stop = end_pos
        self.bus_end_pos = end_stop
        self.bus_end_time = end_time

In [ ]:
# get taxi initial points
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

initial_points = []
sql = '''
select st_transform(initial_point, 3763)
from taxi_services 
'''
cursor_psql.execute(sql)
results = cursor_psql.fetchall()

for row in results:
    initial_points.append(row[0])
print(initial_points[:5])

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# get the closest bus stop for each initial point
initial_stops = []
for initial_point in initial_points[:5]:
    sql = f'''
    select 
        stop_id,
        st_transform(t.initial_point, 3763) as ip,
        st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763),
        st_transform(t.final_point, 3763),
        t.initial_ts
    from taxi_services t
    cross join stops s
    --where st_transform(t.initial_point, 3763) = '{initial_point}'
    order by st_distance(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763), '{initial_point}')
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    for row in results:
        initial_stops.append((row[0], row[2], row[3], row[4]))
print(initial_stops[:5])

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

stop_ids = []

# todos os autocarros que param nessa initial_stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    select trip_id from stop_times where stop_id = '{initial_stop[0]}'
    '''
    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    stop_ids.append(results[0])
print(stop_ids[:5])

In [ ]:
conn = psycopg2.connect("dbname=leo user=leo")
cursor_psql = conn.cursor()

# now we get the final stop
for initial_stop in initial_stops[:100]:
    sql = f'''
    with t2 as ( SELECT DISTINCT ON (t.route_id) t.route_id, st.trip_id, st.departure_time, '{datetime.fromtimestamp(initial_stop[3]).time()}', st.stop_id
                    FROM stop_times st
                    JOIN trips t ON st.trip_id = t.trip_id
                    WHERE st.stop_id = '{initial_stop[0]}'
                      AND st.departure_time > '{datetime.fromtimestamp(initial_stop[3]).time()}'
                    ORDER BY t.route_id, extract(epoch from (st.departure_time - '{datetime.fromtimestamp(initial_stop[3]).time()}')))

    select st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}') as p,
    t2.route_id, 
    t2.trip_id,
    s.stop_id
    from t2 
    join stops s on s.stop_id = t2.stop_id
    group by t2.route_id, t2.trip_id, s.stop_id
    order by st_distance(st_closestpoint(st_collect(st_transform(st_setsrid(st_point(s.stop_lon, s.stop_lat), 4326),3763)), '{initial_stop[2]}'), '{initial_stop[2]}')
    limit 1
    '''

    cursor_psql.execute(sql)
    results = cursor_psql.fetchall()
    print(results, "\n")

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()


# complete query but slow
sql = '''
select 
    t.id,
    s1.stop_id,
    s2.stop_id,
    st_astext(st_transform(t.initial_point, 3763)) as initial_point,
    st_astext(st_transform(t.final_point, 3763)) as final_point,
    st_astext(st_transform(st_setsrid(st_point(s1.stop_lon, s1.stop_lat), 4326), 3763)) as stop_point_init,
    st_astext(st_transform(st_setsrid(st_point(s2.stop_lon, s2.stop_lat), 4326), 3763)) as stop_point_end,
    t.initial_ts,
    t.final_ts,
    s2.departure_time, 
    --no departure time for last stop yet (need a new join this stop_times and that is slow :( )
    s2.trip_id --to later find the final deprature time
from taxi_services t
JOIN LATERAL ( --get init stop_id, s1.stop_id
    SELECT 
        stop_id,
        stop_lon,
        stop_lat
    FROM 
        stops s
    ORDER BY 
        ST_Distance(
            ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763), 
            ST_Transform(t.initial_point, 3763)
        )
    LIMIT 1
) AS s1 ON TRUE
JOIN LATERAL ( --get final stop_id, s2.stop_id
    WITH t2 AS ( --get all trip_ids that pass the init point, only the first trip after intiial_ts from each route will be selected
        SELECT DISTINCT ON (ts.route_id) 
            st.departure_time,
            st.trip_id,
            st.stop_id
        FROM 
            stop_times st
        JOIN 
            trips ts ON st.trip_id = ts.trip_id 
        WHERE 
            st.stop_id = s1.stop_id
            AND 
            st.departure_time > CAST(to_timestamp(t.initial_ts) AS TIME)
        ORDER BY 
            ts.route_id, 
            EXTRACT(EPOCH FROM (st.departure_time - CAST(to_timestamp(t.initial_ts) AS TIME)))
    )
    SELECT --get all point stops that pass on the route
        st.departure_time,
        t2.trip_id,
        --t2.stop_id as init_stop_id,
        st.stop_id,
        s.stop_lat,
        s.stop_lon
    FROM 
        t2 
    JOIN 
        stop_times st ON st.trip_id = t2.trip_id
    JOIN 
        stops s ON s.stop_id = st.stop_id
    GROUP by 
        st.departure_time,
        t2.trip_id, 
        --t2.stop_id,        
        st.stop_id,        
        s.stop_lat,
        s.stop_lon
    ORDER BY 
        ST_Distance(
            ST_ClosestPoint(
                ST_Collect(ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763)),
                ST_Transform(t.final_point, 3763)
            ), 
            ST_Transform(t.final_point, 3763)
        )
    LIMIT 1
) AS s2 ON TRUE
LIMIT 250000;
'''

cursor_psql.execute(sql)
results = cursor_psql.fetchall()
#print(len(results), "\n")
#print(results)

with open('results.txt', 'w+') as f:
    print(results, file=f)


In [ ]:
#build a dictionary of the type dic[route_id] = [trip_id] 
trips_in_routes = {}
#build a dictionary of the type dic[trip_id][stop_id] = departure_time
trip_stop_times = {}


sql_get_trips_from_routes = ''' 
SELECT
    route_id, 
    trip_id
FROM trips t;
'''
cursor_psql.execute(sql_get_trips_from_routes)
results = cursor_psql.fetchall()

for r in results:
    if trips_in_routes.get(r[0]) == None:
        trips_in_routes[r[0]] = []
    trips_in_routes[r[0]].append(r[1])

#-------------------------------------------------

sql_get_stop_times = ''' 
    SELECT trip_id, stop_id, departure_time 
    FROM stop_times;
'''

cursor_psql.execute(sql_get_stop_times)
results = cursor_psql.fetchall()

for r in results:
    if trip_stop_times.get(r[0]) == None:
        trip_stop_times[r[0]] = {}
    trip_stop_times[r[0]][r[1]] = r[2]

for i in trips_in_routes['4M']:
    print(i)
    if(trip_stop_times[i].get('ADA4') != None):
        print(trip_stop_times[i]['ADA4'])
    if(trip_stop_times[i].get('BLG1') != None):
        print(trip_stop_times[i]['BLG1'])

In [ ]:
#quick query fixed
import psycopg2

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()

sql = ''' 
SELECT
    t.id,
    s.route_id,
    s.init_stop_id,
    s.end_stop_id,
    st_astext(st_transform(t.initial_point, 3763)) as initial_point,
    st_astext(st_transform(t.final_point, 3763)) as final_point,
    st_astext(st_transform(st_setsrid(st_point(s.init_lon, s.init_lat), 4326), 3763)) as stop_point_init,
    st_astext(st_transform(st_setsrid(st_point(s.end_lon, s.end_lat), 4326), 3763)) as stop_point_end,
    t.initial_ts,
    t.final_ts
FROM taxi_services t
JOIN LATERAL (
    WITH p1 AS (
        SELECT 
            stop_id
        FROM 
            stops s
        ORDER BY 
            ST_Distance(
                ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763), 
                ST_Transform(t.initial_point, 3763)
            )
        LIMIT 5
    ),
    p2 AS (
        SELECT DISTINCT ON (rs.route_id, rs.direction_id)
            rs.route_id,
            rs.direction_id,
            rs.stop_sequence,
            p1.stop_id,
            rs.stop_lon,
            rs.stop_lat
        FROM 
            route_stops rs
        JOIN p1 
        ON rs.stop_id = p1.stop_id
        GROUP BY
            rs.route_id,
            rs.direction_id,
            rs.stop_sequence,
            p1.stop_id,
            rs.stop_lon,
            rs.stop_lat
    )
    SELECT
        p2.stop_id as init_stop_id,
        p2.stop_lat as init_lat,
        p2.stop_lon as init_lon,
        rs.stop_id as end_stop_id,
        rs.stop_lat as end_lat,
        rs.stop_lon as end_lon,
        rs.route_id
    FROM
        route_stops rs
    JOIN
        p2
    ON rs.route_id = p2.route_id
    WHERE
        p2.direction_id = rs.direction_id
        AND p2.stop_sequence < rs.stop_sequence
    GROUP BY 
        p2.stop_id,
        p2.stop_lat,
        p2.stop_lon,
        rs.stop_id,
        rs.stop_lat,
        rs.stop_lon,
        rs.route_id
    ORDER BY 
        ST_Distance(
            ST_ClosestPoint(
                ST_Collect(ST_Transform(ST_SetSRID(ST_Point(rs.stop_lon, rs.stop_lat), 4326), 3763)),
                ST_Transform(t.final_point, 3763)
            ), 
            ST_Transform(t.final_point, 3763)
        )
        --ordenar pela soma das distancias
        
    LIMIT 1
) AS s ON TRUE
LIMIT 10000
'''
cursor_psql.execute(sql)
results = []
results = cursor_psql.fetchall()
print(len(results), "\n")
#print(results)

In [ ]:
#for r in results[:5]:
for r in results:
    #print(r)
    t_init = datetime.fromtimestamp(r[7]).time()
    trip = ''
    best_time = -1
    best_time_found = False
    print(r[9])
    for t in trips_in_routes[r[9]]:
        #print(t, r[9], r[1], trip_stop_times.get(t), trip_stop_times[t].get(r[1]))
        if(trip_stop_times.get(t) == None): continue
        if(trip_stop_times[t].get(r[1]) == None): continue
        if(trip_stop_times[t].get(r[2]) == None): continue

        if(trip_stop_times[t][r[1]] > t_init):
            if(best_time_found):
                if trip_stop_times[t][r[1]] < best_time:
                    best_time = trip_stop_times[t][r[1]]
                    trip = t
            else:
                best_time = trip_stop_times[t][r[1]]
                best_time_found = 1
                trip = t
    t_end = -1
    if(trip != ''):
        t_end = trip_stop_times[trip][r[2]]
    print("trip id:", r[0], end = " ")
    print("trip:", trip, end = " ")
    print("init stop:", r[1], end = " ")
    print("end stop:", r[2], end = " ")
    print("init time:", best_time, end = " ")
    print("end time:", t_end)

    

In [ ]:
#print(trip_stop_times['503_0_V_47'])
for t in trips_in_routes['703']:
    print(t,len(trip_stop_times[t]))

In [ ]:
st_astext(ST_Transform(ST_SetSRID(ST_Point(stop_lon, stop_lat), 4326), 3763))

In [ ]:
import psycopg2
import matplotlib.pyplot as plt
from datetime import datetime


class Trip:
    def __init__(self, route_id, bus_init_stop, bus_end_stop, taxi_init_pos, taxi_end_pos, bus_init_pos, bus_end_pos, taxi_init_time, taxi_end_time):
        self.route_id = route_id
        
        self.bus_init_stop = bus_init_stop
        self.bus_end_stop = bus_end_stop

        self.taxi_init_pos = taxi_init_pos
        self.taxi_end_pos = taxi_end_pos



        self.bus_init_pos = bus_init_pos
        self.bus_end_pos = bus_end_pos

        self.taxi_init_time = taxi_init_time
        self.taxi_end_time = taxi_end_time

        self.trip_id = -1

    def set_times(self, trip_id, init_time, end_time):
        self.trip_id = trip_id
        self.bus_init_time = init_time
        self.bus_end_time = end_time

    def __str__(self):
        return "insert into warehouse values ( {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {} )".format(
            self.route_id,
            self.trip_id,
            self.bus_init_stop,
            self.bus_end_stop, 
            self.taxi_init_time,
            self.taxi_end_time,
            self.bus_init_time,
            self.bus_end_time,
            self.taxi_init_pos,
            self.taxi_end_pos,
            self.bus_init_pos,
            self.bus_end_pos
        )

#-------------------------------------------------
#-------------------------------------------------
#-------------------------------------------------

#connection

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()

#-------------------------------------------------
#-------------------------------------------------
#-------------------------------------------------

#build a dictionary of the type dic[route_id] = [trip_id] 
trips_in_routes = {}

sql_get_trips_from_routes = ''' 
SELECT
    route_id, 
    trip_id
FROM trips t
WHERE service_id = 'UTEIS'
'''
cursor_psql.execute(sql_get_trips_from_routes)
results = cursor_psql.fetchall()

for r in results:
    if trips_in_routes.get(r[0]) == None:
        trips_in_routes[r[0]] = []
    trips_in_routes[r[0]].append(r[1])

#-------------------------------------------------

#build a dictionary of the type dic[trip_id][stop_id] = departure_time
trip_stop_times = {}

sql_get_stop_times = ''' 
    SELECT st.trip_id, stop_id, departure_time 
    FROM stop_times st
    JOIN trips t
    ON st.trip_id = t.trip_id
    WHERE t.service_id = 'UTEIS'
    ;
'''

cursor_psql.execute(sql_get_stop_times)
results = cursor_psql.fetchall()

for r in results:
    if trip_stop_times.get(r[0]) == None:
        trip_stop_times[r[0]] = {}
    if trip_stop_times[r[0]].get(r[1]) == None:
        trip_stop_times[r[0]][r[1]] = []
    trip_stop_times[r[0]][r[1]].append(r[2])

#-------------------------------------------------
#-------------------------------------------------
#-------------------------------------------------
    
main_query = ''' 
SELECT
    --t.id,
    s.route_id,
    s.init_stop_id,
    s.end_stop_id,
    st_astext(st_transform(t.initial_point, 3763)) as initial_point,
    st_astext(st_transform(t.final_point, 3763)) as final_point,
    st_astext(st_transform(st_setsrid(st_point(s.init_lon, s.init_lat), 4326), 3763)) as stop_point_init,
    st_astext(st_transform(st_setsrid(st_point(s.end_lon, s.end_lat), 4326), 3763)) as stop_point_end,
    CAST(to_timestamp(t.initial_ts) AS TIME),
    CAST(to_timestamp(t.final_ts) AS TIME)
FROM taxi_services t
JOIN LATERAL (
    WITH p1 AS ( -- get initial k closets points to initial_point
        SELECT 
            stop_id
        FROM 
            stops s
        ORDER BY 
            ST_Distance(
                ST_Transform(ST_SetSRID(ST_Point(s.stop_lon, s.stop_lat), 4326), 3763), 
                ST_Transform(t.initial_point, 3763)
            )
        LIMIT 5
    ),
    p2 AS ( -- get all points from the routes that pass on all initial points p1
        --SELECT DISTINCT ON (rs.route_id, rs.direction_id, rs.max_sequence, rs.trip_headsign)
        SELECT DISTINCT ON (rs.route_id, rs.direction_id)
            rs.route_id,
            rs.direction_id,
            rs.stop_sequence,
            rs.max_sequence,
            rs.trip_headsign,
            p1.stop_id,
            rs.stop_lon,
            rs.stop_lat
        FROM 
            route_stops rs
        JOIN p1 
        ON rs.stop_id = p1.stop_id
        WHERE CAST(to_timestamp(t.initial_ts) AS TIME) <= rs.last_departure
        --GROUP BY
            --rs.route_id,
            --rs.direction_id,
            --rs.stop_sequence,
            --rs.max_sequence,
            --rs.trip_headsign,
            --p1.stop_id,
            --rs.stop_lon,
            --rs.stop_lat
    )
    SELECT
        p2.stop_id as init_stop_id,
        p2.stop_lat as init_lat,
        p2.stop_lon as init_lon,
        rs.stop_id as end_stop_id,
        rs.stop_lat as end_lat,
        rs.stop_lon as end_lon,
        rs.route_id
    FROM
        route_stops rs
    JOIN
        p2
    ON rs.route_id = p2.route_id 
    WHERE
        p2.direction_id = rs.direction_id
        AND p2.stop_sequence > rs.stop_sequence
        --AND p2.max_sequence = rs.max_sequence
        --AND p2.trip_headsign = rs.trip_headsign
    GROUP BY 
        p2.stop_id,
        p2.stop_lat,
        p2.stop_lon,
        rs.stop_id,
        rs.stop_lat,
        rs.stop_lon,
        rs.route_id
    ORDER BY 
        ST_Distance(
            ST_ClosestPoint(
                ST_Collect(ST_Transform(ST_SetSRID(ST_Point(rs.stop_lon, rs.stop_lat), 4326), 3763)),
                ST_Transform(t.final_point, 3763)
            ), 
            ST_Transform(t.final_point, 3763)
        )
        --ordenar pela soma das distancias
        
    LIMIT 1
) AS s ON TRUE
LIMIT 1;
'''

main_query = ''' 
SELECT
    s.route_id,
    s.init_stop_id,
    s.end_stop_id,
    ST_AsText(ST_Transform(t.initial_point, 3763)) AS initial_point,
    ST_AsText(ST_Transform(t.final_point, 3763)) AS final_point,
    ST_AsText(ST_Transform(ST_SetSRID(ST_Point(s.init_lon, s.init_lat), 4326), 3763)) AS stop_point_init,
    ST_AsText(ST_Transform(ST_SetSRID(ST_Point(s.end_lon, s.end_lat), 4326), 3763)) AS stop_point_end,
    CAST(TO_TIMESTAMP(t.initial_ts) AS TIME),
    CAST(TO_TIMESTAMP(t.final_ts) AS TIME)
FROM taxi_services t
JOIN LATERAL (
    WITH p1 AS (
        SELECT 
            stop_id,
            stop_lat,
            stop_lon
        FROM 
            stops
        ORDER BY 
            ST_Distance(
                ST_Transform(ST_SetSRID(ST_Point(stop_lon, stop_lat), 4326), 3763), 
                ST_Transform(t.initial_point, 3763)
            )
        LIMIT 5
    ),
    p2 AS (
        SELECT DISTINCT ON (rs.route_id, rs.direction_id)
            rs.route_id,
            rs.direction_id,
            rs.stop_sequence,
            rs.max_sequence,
            rs.trip_headsign_hash,
            p1.stop_id,
            rs.stop_lon,
            rs.stop_lat
        FROM 
            route_stops rs
        JOIN p1 
        ON rs.stop_id = p1.stop_id
        WHERE CAST(TO_TIMESTAMP(t.initial_ts) AS TIME) <= rs.last_departure
    ),
    s AS (
        SELECT
            p2.stop_id AS init_stop_id,
            p2.stop_lat AS init_lat,
            p2.stop_lon AS init_lon,
            rs.stop_id AS end_stop_id,
            rs.stop_lat AS end_lat,
            rs.stop_lon AS end_lon,
            rs.route_id
        FROM
            route_stops rs
        JOIN
            p2
        ON rs.route_id = p2.route_id 
        WHERE
            p2.direction_id = rs.direction_id
            AND p2.trip_headsign_hash = rs.trip_headsign_hash
            AND p2.stop_sequence < rs.stop_sequence
            AND p2.max_sequence = rs.max_sequence
            AND CAST(TO_TIMESTAMP(t.final_ts) AS TIME) <= rs.last_departure
        ORDER BY 
            ST_Distance(
                ST_Transform(ST_SetSRID(ST_Point(rs.stop_lon, rs.stop_lat), 4326), 3763),
                ST_Transform(t.final_point, 3763)
            )
        LIMIT 1
    )
    SELECT
        s.route_id,
        s.init_stop_id,
        s.end_stop_id,
        s.init_lat,
        s.init_lon,
        s.end_lat,
        s.end_lon
    FROM s

) AS s ON TRUE
LIMIT 100000;
'''

results = []
cursor_psql.execute(main_query)
results = cursor_psql.fetchall()

#-------------------------------------------------
#-------------------------------------------------
#-------------------------------------------------

#create trips array
trips = []

for r in results:
    trips.append(Trip(*r)) # * unpacks the array

#-------------------------------------------------
#-------------------------------------------------
#-------------------------------------------------

#get stop times

failed_data_points = 0

for t in trips:
    trip = ''
    best_time_found = False
    best_init = -1
    best_end = -1

    #print(t.taxi_init_time)
    t_init = t.taxi_init_time
    for tr in trips_in_routes[t.route_id]:
        if(trip_stop_times.get(tr) == None): continue
        if(trip_stop_times[tr].get(t.bus_init_stop) == None): continue
        if(trip_stop_times[tr].get(t.bus_end_stop) == None): continue


        for init in trip_stop_times[tr][t.bus_init_stop]:
            for end in trip_stop_times[tr][t.bus_end_stop]:
                #print(init, end)
                if(init > t_init):
                    if(init < end):
                        if(best_time_found):
                            if init < best_init and end < best_end:
                            #if end < best_end:
                                best_init = init
                                best_end = end
                                trip = tr
                        else:
                            best_init = init
                            best_end = end
                            best_time_found = 1
                            trip = tr


    if(trip != ''):
        if(best_end < best_init):
            print("error", trip, best_init, best_end)
            print("trip time", t_init, "begin stop:", t.bus_init_stop, "end stop:", t.bus_end_stop, "route", t.route_id)
            failed_data_points += 1
            continue
        t.set_times(trip, best_init, best_end)
    else:
        print("trip time", t_init, "begin stop:", t.bus_init_stop, "end stop:", t.bus_end_stop, "route", t.route_id, "trip", t.trip_id)
        failed_data_points += 1

trips = [t for t in trips if t.trip_id != -1]

#-------------------------------------------------
#-------------------------------------------------
#-------------------------------------------------

#create sql file with all rows
    
with open("fill_warehouse.sql", "w+") as f:
    for t in trips:
        print(t, file=f)

#for t in trips:
    #print(t)

print("failed entries", failed_data_points)

In [ ]:
print(trip_stop_times['302_0_U_2'])

In [ ]:
import psycopg2

conn = psycopg2.connect(dbname = 'stcp', user = 'aluno', password = 'aluno')
cursor_psql = conn.cursor()

In [ ]:
    ,s AS (
        SELECT
            p2.stop_id AS init_stop_id,
            p2.stop_lat AS init_lat,
            p2.stop_lon AS init_lon,
            rs.stop_id AS end_stop_id,
            rs.stop_lat AS end_lat,
            rs.stop_lon AS end_lon,
            rs.route_id
        FROM
            route_stops rs
        JOIN
            p2
        ON rs.route_id = p2.route_id 
        WHERE
            p2.direction_id = rs.direction_id
            AND p2.stop_sequence < rs.stop_sequence
            AND p2.max_sequence = rs.max_sequence
            AND p2.trip_headsign_hash = rs.trip_headsign_hash
        ORDER BY 
            ST_Distance(
                ST_Transform(ST_SetSRID(ST_Point(rs.stop_lon, rs.stop_lat), 4326), 3763),
                ST_Transform(t.final_point, 3763)
            )
        LIMIT 1
    )
    SELECT
        s.route_id,
        s.init_stop_id,
        s.end_stop_id,
        s.init_lat,
        s.init_lon,
        s.end_lat,
        s.end_lon
    FROM s